In [1]:
import pandas as pd



train_log_filename = r'../train/train_log.csv'
test_log_filename = r'../test/test_log.csv'

train_log = pd.read_csv(train_log_filename, sep='\t')

test_log = pd.read_csv(test_log_filename, sep='\t')

df_log = pd.concat([train_log, test_log]).reset_index(drop=True)

df_log['OCC_TIM'] = pd.to_datetime(df_log['OCC_TIM'])


In [2]:
df_log['day']  =df_log['OCC_TIM'].dt.day
df_log['hour']  =df_log['OCC_TIM'].dt.hour
df_log['minute']  =df_log['OCC_TIM'].dt.minute

In [3]:
t1=df_log.groupby(['USRID','day'],as_index=False)['day'].agg({'click_count':len})

In [4]:
tmp = pd.pivot_table(t1,index=['USRID'],columns=['day'],values=['click_count'],fill_value=0,margins=False)
s= tmp['click_count']
s.columns = s.columns.map(lambda x:'day'+str(x)+'click_count')
s.columns.name=None
s = s.reset_index()


In [5]:
s.to_csv('../final_feature/day_click_count.csv',index=None,sep='\t')

In [6]:
t1 = t1.sort_values(by=['USRID','day'])

In [7]:
import numpy as np
t1['next_day'] = t1.groupby(['USRID'])['day'].diff(-1).apply(np.abs)
log1 = t1.groupby(['USRID'],as_index=False)['next_day'].agg({
    'next_day_mean':np.mean,
    'next_day_std':np.std,
    'next_day_min':np.min,
    'next_day_max':np.max
})

In [8]:
log1.head() #时间 以天数 为间隔单位的  平均值，方差，最小最大。

,USRID,next_day_mean,next_day_std,next_day_min,next_day_max
0,2,1.000,NaN,1.0,1.0
1,3,3.125,2.948971,1.0,10.0
2,4,NaN,NaN,NaN,NaN
3,5,7.500,0.707107,7.0,8.0
4,7,4.750,4.349329,1.0,11.0


In [9]:
t1.head()
log2 = t1.groupby(['USRID'],as_index=False)['click_count'].agg({
    'day_click_count_mean':np.mean,
    'day_click_count_std':np.std,
    'day_click_count_min':np.min,
   'day_click_count_max':np.max
})

In [10]:
feature1 = pd.merge(log1,log2)
feature1.to_csv('../final_feature/day_statical.csv',index=None,sep='\t')

In [11]:
df_log.groupby(['USRID','day','hour'],as_index=False)['hour'].agg({'click_count':len})

,USRID,day,hour,click_count
0,2,13,23,8
1,2,14,19,1
2,3,3,9,9
3,3,3,10,5
4,3,3,20,10
5,3,6,13,8
6,3,9,20,6
7,3,12,17,12
8,3,22,10,11
9,3,23,15,21


In [12]:
t1.head()

,USRID,day,click_count,next_day
0,2,13,8,1.0
1,2,14,1,NaN
2,3,3,24,3.0
3,3,6,8,3.0
4,3,9,6,3.0


In [13]:
df_log[df_log['USRID']==4]

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,day,hour,minute
33883,4,326-1041-1678,2018-03-20 12:09:32,0,20,12,9
33884,4,326-1042-1703,2018-03-20 12:11:26,0,20,12,11
33885,4,38-115-117,2018-03-20 12:09:25,0,20,12,9
889553,4,326-1040-1677,2018-03-20 12:09:33,0,20,12,9
889554,4,326-1041-1678,2018-03-20 12:11:24,0,20,12,11
889555,4,520-1836-3640,2018-03-20 12:09:27,0,20,12,9
891067,4,0-231-277,2018-03-20 12:09:57,0,20,12,9
891068,4,326-1042-1703,2018-03-20 12:10:32,0,20,12,10
891069,4,326-1047-1724,2018-03-20 12:10:50,0,20,12,10
891070,4,359-1234-2004,2018-03-20 12:09:33,0,20,12,9


In [14]:
df_log = df_log.sort_values(by=['OCC_TIM'])

import numpy as np
t1=df_log.groupby(['USRID','day','hour'],as_index=False)['hour'].agg({'click_count':len})
t1['day_hour'] = t1['day']*24+t1['hour']

t1['next_hour'] = t1.groupby(['USRID'])['day_hour'].diff(-1).apply(np.abs) #时间差
log1 = t1.groupby(['USRID'],as_index=False)['next_hour'].agg({
    'next_hour_mean':np.mean,
    'next_hour_std':np.std,
    'next_hour_min':np.min,
    'next_hour_max':np.max
}) #小时  ，间隔的时间差

In [15]:



log2 = t1.groupby(['USRID'],as_index=False)['click_count'].agg({
   'hour_click_count_mean':np.mean,
    'hour_click_count_std':np.std,
    'hour_click_count_min':np.min,
   'hour_click_count_max':np.max
}) #小时  作为单位的点击数的统计量

In [16]:
feature2=pd.merge(log1,log2)
feature2.to_csv('../final_feature/hour_statical.csv',index=None,sep='\t')

In [17]:
df_log = df_log.sort_values(by=['OCC_TIM'])

t1=df_log.groupby(['USRID','day','hour','minute'],as_index=False)['minute'].agg({'click_count':len})
t1['true_minute'] = t1['day']*24*60+t1['hour']*60+t1['minute']

t1['next_minute'] = t1.groupby(['USRID'])['true_minute'].diff(-1).apply(np.abs) #时间差
log1 = t1.groupby(['USRID'],as_index=False)['next_minute'].agg({
    'next_minute_mean':np.mean,
    'next_minute_std':np.std,
    'next_minute_min':np.min,
    'next_minute_max':np.max
}) #小时  ，间隔的时间差

In [18]:
log2 = t1.groupby(['USRID'],as_index=False)['click_count'].agg({
   'minute_click_count_mean':np.mean,
    'minute_click_count_std':np.std,
    'minute_click_count_min':np.min,
   'minute_click_count_max':np.max
}) #小时  ，间隔的时间

In [19]:
feature2=pd.merge(log1,log2)
feature2.to_csv('../final_feature/minute_statical.csv',index=None,sep='\t')

In [20]:
df_log['weekday']  =df_log['OCC_TIM'].dt.weekday
t1 =df_log.groupby(['USRID','weekday',],as_index=False)['weekday'].agg({'click_count':len})

In [21]:
tmp = pd.pivot_table(t1,index=['USRID'],columns=['weekday'],values=['click_count'],fill_value=0,margins=False)
s= tmp['click_count']
s.columns = s.columns.map(lambda x:'weekday'+str(x)+'click_count')
s.columns.name=None
s = s.reset_index()

In [22]:
s.head()

,USRID,weekday0click_count,weekday1click_count,weekday2click_count,weekday3click_count,weekday4click_count,weekday5click_count,weekday6click_count
0,2,0,8,1,0,0,0,0
1,3,32,33,30,11,27,24,0
2,4,0,18,0,0,0,0,0
3,5,2,18,0,0,0,0,0
4,7,0,8,44,0,16,8,0


In [23]:
s['weekendclick_count']=s['weekday0click_count']+s['weekday6click_count']
s['notweekendclick_count']=s['weekday1click_count']+s['weekday2click_count']+s['weekday3click_count']+s['weekday4click_count']+s['weekday5click_count']

In [24]:
s.to_csv('../final_feature/weekday_click_count.csv',index=None,sep='\t')

In [25]:
df_log['weekday'] = df_log['OCC_TIM'].dt.weekday


In [26]:
df_log

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,day,hour,minute,weekday
762645,35437,162-555-891,2018-03-01 00:00:00,0,1,0,0,3
797071,35437,163-577-913,2018-03-01 00:00:00,0,1,0,0,3
729470,35437,162-555-891,2018-03-01 00:00:01,0,1,0,0,3
1536943,35437,163-577-913,2018-03-01 00:00:02,0,1,0,0,3
762646,35437,162-555-891,2018-03-01 00:00:03,0,1,0,0,3
3617939,14879,326-1041-1678,2018-03-01 00:00:03,0,1,0,0,3
3189637,35437,163-577-913,2018-03-01 00:00:04,0,1,0,0,3
3621268,14879,326-1043-1704,2018-03-01 00:00:04,0,1,0,0,3
3366379,58718,520-1841-3766,2018-03-01 00:00:05,0,1,0,0,3
3574568,14879,438-1484-2644,2018-03-01 00:00:06,0,1,0,0,3


In [27]:
def  fun1(x):
    if x[7]==3:#周三下午3点茶
        if x[5]==14:
            if x[6]>=50:
                return 1
        elif(x[5]==15):
            if x[6]<=10:
                return 1
        else:
            return 0
    return 0
df_log['promote1']=df_log.apply(fun1,axis=1)

In [28]:
def  fun2(x):
    
    if x[5]==9:
        if x[6]>=50:
            return 1
    elif(x[5]==10):
        if x[6]<=10:
            return 1
    else:
        return 0
    
df_log['promote2']=df_log.apply(fun1,axis=1)#2个促销时间

In [29]:
#计算最后一次登陆  距离4.1号的时间
df_log['day_4_1'] = 32-df_log['day']

t1=df_log[['USRID','day_4_1']].drop_duplicates().groupby(['USRID'],as_index=False)['day_4_1'].agg({'dist_41_min':np.min,'dist_41_max':np.max,'dist_41_std':np.std,'dist_41_mean':np.mean})


In [30]:
t2 = df_log.groupby(['USRID'])['promote1'].agg({'promote1':sum})
t3 = df_log.groupby(['USRID'])['promote2'].agg({'promote2':sum})
t=pd.merge(t1,t2,on='USRID')
t=pd.merge(t,t3,on='USRID')


D:\Users\cbf11\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
D:\Users\cbf11\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [31]:
t.to_csv('../final_feature/day_dist_promote.csv',index=None)